In [6]:
# Now universal MACE finetuning on T2
#!/usr/bin/env python3
import os
import subprocess
import sys

def main():
    # ——— Environment setup ———
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    cmd = [
        "mace_run_train",
        # === General settings ===
        "--name",              "mace_T2_including_replay_w2",
        "--model",             "MACE",
        "--num_interactions",  "2",
        "--foundation_model",  "/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model",
        "--foundation_model_readout",
    # --- MP replay (pretraining head) ---
        # "--pt_train_file","/home/phanim/harshitrawat/summer/replay_data/mp_finetuning-mace_T2_mp_replay_run-42.xyz",              # <- MP replay shortcut
        # "--atomic_numbers","[3,8,40,57]",    # Li, O, Zr, La
        # "--multiheads_finetuning","True",

        "--train_file","/home/phanim/harshitrawat/summer/iteration_3-copy/one.xyz",
        "--valid_fraction", "0.5",
        "--batch_size",        "2",
        "--valid_batch_size",  "1",

        "--device",            "cuda",

        # === Loss function weights ===
        "--forces_weight",     "0",         # Increased force weight to balance energy better
        "--energy_weight",     "10",   
        "--stress_weight", "0",             # Reduced from 100 → avoid dominance + stabilize energy RMSE

        # === Learning setup ===
        "--lr",                "0.006",      # Explicit learning rate (0.0001 is too low → stagnation)
        "--scheduler_patience","4",          # Reduce LR if val loss doesn’t improve in 3 epochs
        "--clip_grad",         "1",        # Avoid exploding gradients — essential when energy_weight is high
        "--weight_decay",      "1e-8",       # Mild regularization to prevent overfitting

        # === EMA helps smooth loss curve ===
        #"--ema_decay",         "0.999",     # Smooths validation loss and helps final convergence

        # === Domain + training settings ===
        "--r_max",             "5.0",
        "--max_num_epochs",    "77",
        "--E0s",               "{3: -1.2302615750354944, 8: -23.049110738413006, 40: 23.367646191010394, 57: 15.192898072498549}",    # Still allowed — could optionally be replaced by manual E0s
        "--seed",              "84",
        "--patience",     "8",

        "--restart_latest",                   # Resumes from checkpoint if available
    ]

    print("Running:", " \\\n    ".join(cmd), file=sys.stderr)
    subprocess.run(cmd, check=True)

if __name__ == "__main__":
    main()


Running: mace_run_train \
    --name \
    mace_T2_including_replay_w2 \
    --model \
    MACE \
    --num_interactions \
    2 \
    --foundation_model \
    /home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model \
    --foundation_model_readout \
    --train_file \
    /home/phanim/harshitrawat/summer/iteration_3-copy/one.xyz \
    --valid_fraction \
    0.5 \
    --batch_size \
    2 \
    --valid_batch_size \
    1 \
    --device \
    cuda \
    --forces_weight \
    0 \
    --energy_weight \
    10 \
    --stress_weight \
    0 \
    --lr \
    0.006 \
    --scheduler_patience \
    4 \
    --clip_grad \
    1 \
    --weight_decay \
    1e-8 \
    --r_max \
    5.0 \
    --max_num_epochs \
    77 \
    --E0s \
    {3: -1.2302615750354944, 8: -23.049110738413006, 40: 23.367646191010394, 57: 15.192898072498549} \
    --seed \
    84 \
    --patience \
    8 \
    --restart_latest
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packag

2025-08-21 20:38:33.809 INFO: ===========VERIFYING SETTINGS===========
2025-08-21 20:38:33.809 INFO: MACE version: 0.3.14
2025-08-21 20:38:33.878 INFO: CUDA version: 12.6, CUDA device: 0


/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/cli/run_train.py:152: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  model_foundation = torch.load(


2025-08-21 20:38:34.433 INFO: Using foundation model /home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model as initial checkpoint.
2025-08-21 20:38:34.434 WARNING: Using multiheads finetuning with a foundation model that is not a Materials Project model, need to provied a path to a pretraining file with --pt_train_file.
2025-08-21 20:38:34.434 INFO: ===========LOADING INPUT DATA===========
2025-08-21 20:38:34.434 INFO: Using heads: ['Default']
2025-08-21 20:38:34.434 INFO: Using the key specifications to parse data:
2025-08-21 20:38:34.434 INFO: Default: KeySpecification(info_keys={'energy': 'REF_energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head', 'elec_temp': 'elec_temp', 'total_charge': 'total_charge', 'polarizability': 'polarizability', 'total_spin': 'total_spin'}, arrays_keys={'forces': 'REF_forces', 'charges': 'REF_charges'})
2025-08-21 20:38:34.434 INFO: =============    Processing head Default     

Traceback (most recent call last):
  File "/home/phanim/harshitrawat/miniconda3/bin/mace_run_train", line 8, in <module>
    sys.exit(main())
             ~~~~^^
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/cli/run_train.py", line 77, in main
    run(args)
    ~~~^^^^^^
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/cli/run_train.py", line 683, in run
    args.avg_num_neighbors = get_avg_num_neighbors(head_configs, args, train_loader, device)
                             ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/tools/scripts_utils.py", line 551, in get_avg_num_neighbors
    avg_num_neighbors = modules.compute_avg_num_neighbors(train_loader)
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/modules/utils.py", line 414, in compute_avg_num_neighbors
    torch.cat(num_neighbors, dim=0).type(torch

CalledProcessError: Command '['mace_run_train', '--name', 'mace_T2_including_replay_w2', '--model', 'MACE', '--num_interactions', '2', '--foundation_model', '/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model', '--foundation_model_readout', '--train_file', '/home/phanim/harshitrawat/summer/iteration_3-copy/one.xyz', '--valid_fraction', '0.5', '--batch_size', '2', '--valid_batch_size', '1', '--device', 'cuda', '--forces_weight', '0', '--energy_weight', '10', '--stress_weight', '0', '--lr', '0.006', '--scheduler_patience', '4', '--clip_grad', '1', '--weight_decay', '1e-8', '--r_max', '5.0', '--max_num_epochs', '77', '--E0s', '{3: -1.2302615750354944, 8: -23.049110738413006, 40: 23.367646191010394, 57: 15.192898072498549}', '--seed', '84', '--patience', '8', '--restart_latest']' returned non-zero exit status 1.

✅ Keys renamed to REF_* and saved to:
/home/phanim/harshitrawat/summer/replay_data/mp_finetuning-mace_T2_mp_replay_run-42_REFkeys.xyz


✅ Header updated:
→ /home/phanim/harshitrawat/summer/replay_data/mp_finetuning-mace_T2_mp_replay_run-42_REFkeys.xyz
